# Azure AI Agent service - Bing custom search

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingCustomSearchTool, MessageRole
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 14-Jun-2025 10:08:55


## 1. Grounding with Bing Custom Search

### Connect to AI Foundry

In [5]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to custom bing grounding service, the config only has ARXIV.ORG in it !!

In [6]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("BING_CUSTOM_CONNECTION_ID")

# Initialize Bing Custom Search tool with connection id and instance name
bing_custom_tool = BingCustomSearchTool(connection_id=conn_id, instance_name="first-config")

### Create agent with Bing Grounding tool

In [7]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-bing-grounding-agent",
        instructions=instructions,
        tools=bing_custom_tool.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

bing_custom_tool
bing_custom_tool.definitions


Created agent, ID: asst_khh99qArWzQyn0YAmcUKHCkz


[{'type': 'bing_custom_search', 'bing_custom_search': {'search_configurations': [{'connection_id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqbingcustomsearch002', 'instance_name': 'first-config', 'market': '', 'set_lang': '', 'count': 5, 'freshness': ''}]}}]

### Create THREAD

In [8]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_g60O2rvn3sOAccchz7ErxbLb


### Example #1

In [9]:
query = "what is are the latest news on AI?"

### Create first message in the thread

In [10]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_LBzlIBFCe6RGeD8RS1LDINoL


### Create first RUN

In [11]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


### Check status of the RUN and tool calls

In [12]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage


Step step_7Zxc9KdvYrIab3lMq8f8wLzo status: completed
Step step_IvxdhiUR1Ip5YoEmNfdZ1nqV status: completed
  Tool calls:
    Tool Call ID: call_shFf1XIJpvkOYgEZtfbmlTpR
    Type: bing_custom_search



{'prompt_tokens': 10703, 'completion_tokens': 325, 'total_tokens': 11028, 'prompt_token_details': {'cached_tokens': 0}}

### Get response

In [13]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The latest news in AI as of June 2025 highlights several significant trends and developments:

1. **Artificial Intelligence Index Report 2025**: This comprehensive report details the rapid advancement and growing impact of AI in society, the economy, and global governance. New in this edition are in-depth analyses of AI hardware, inference costs, as well as trends in AI publications and patents. The report also covers the adoption of responsible AI practices in corporations and the expanding role of AI in science and medicine—serving as a resource for policymakers, researchers, and the public to better understand the state and direction of AI globally【3:0†source】.

2. **arXiv AI Research**: Recent submissions on arXiv show continuous explorations at the frontier of AI, including:
   - Objective-driven algorithm engineering,
   - Multi-agent cooperation via reinforcement learning,
   - Evaluation and benchmarking of language models, especially on their failure modes and 

### Example #2

In [14]:
query = "find out about the latest iPhone"

In [15]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_BqtSCisRGo469KGovdxCafZu


In [16]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [17]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_yE5YlDZGxJtXTogbKVlVbHT9 status: completed
Step step_CcaC9TRVd8tEJ4znsh4lEdCT status: completed
  Tool calls:
    Tool Call ID: call_XRkVbvWfbCLzfjuzkeJvtlZk
    Type: bing_custom_search



{'prompt_tokens': 7886, 'completion_tokens': 225, 'total_tokens': 8111, 'prompt_token_details': {'cached_tokens': 0}}

In [18]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The latest iPhone as of June 2025 is expected to be the iPhone 17 series, which continues Apple's annual release cycle. Although the official launch is typically in September each year, leaks and industry news suggest the iPhone 17 will feature significant upgrades, including a new generation of Apple's A-series chip, advancements in camera technology, design refinements, and possible new AI-driven features. There are also rumors of an "Ultra" variant joining the regular lineup, which would focus on high-end photography and pro-level hardware.

Anticipated improvements include thinner bezels, increased battery life, and a further push toward integrating AI into on-device user experiences. Apple is expected to introduce iOS 19 with the new devices, offering enhanced customization, security, and features built on generative AI models to compete with other leading tech companies.

For the most precise and current launch details, Apple's official website or major tech news 

### Example #3

In [19]:
query = "What is the weather informations for Mexico city? Show the report and use emojis and be funny"

In [20]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_9l4ACFkJUq6fQ4CpI0S0UIeA


In [21]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [22]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_SwXToqhN790lBXoDnmgC1KJD status: completed
Step step_CqNGA0MMLUMlrpBiglaJXcBB status: completed
  Tool calls:
    Tool Call ID: call_HntSfUfhvnu7RFQsrKx3STSb
    Type: bing_custom_search



{'prompt_tokens': 3136, 'completion_tokens': 85, 'total_tokens': 3221, 'prompt_token_details': {'cached_tokens': 0}}

In [23]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: Sorry, I'm unable to fetch the current weather for Mexico City right now. For the latest weather updates, I recommend checking a reliable weather app or website like Weather.com or AccuWeather. If you want a fun, emoji-filled weather report in the future, just ask again! 🌞⛅🌩️


### Post processing

In [24]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_khh99qArWzQyn0YAmcUKHCkz, Name: my-bing-grounding-agent, Model: gpt-4.1, Instructions: You are an AI Agent that can do some web search using Bing


In [26]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [27]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")